In [1]:
from typing import List
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import END, StateGraph, MessagesState
from langchain.text_splitter import TokenTextSplitter

# LLMモデルの設定
local_path = "gguf_models/Llama-3-ELYZA-JP-8B-q4_k_m.gguf"
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

llm = LlamaCpp(
    model_path=local_path,
    callback_manager=callback_manager,
    verbose=False
)

# エージェントの状態を管理するクラス
class AgentState(MessagesState):
    agent_name: str
    other_agent_name: str

def create_agent_state(name: str, other_name: str) -> AgentState:
    return AgentState(messages=[], agent_name=name, other_agent_name=other_name)

# トークン数を制限しつつ会話履歴を管理する関数
def get_truncated_conversation_history(messages: List, agent_name: str, other_agent_name: str, max_tokens: int = 400):
    text_splitter = TokenTextSplitter(chunk_size=max_tokens, chunk_overlap=0)
    
    conversation_history = ""
    for message in reversed(messages):
        if isinstance(message, HumanMessage):
            conversation_history = f"{other_agent_name}: {message.content}\n" + conversation_history
        elif isinstance(message, AIMessage):
            conversation_history = f"{agent_name}: {message.content}\n" + conversation_history
    
    truncated_history = text_splitter.split_text(conversation_history)[-1]
    return truncated_history

# エージェントの応答を生成する関数

# def generate_response(state: AgentState):
#     truncated_history = get_truncated_conversation_history(
#         state['messages'], 
#         state['agent_name'], 
#         state['other_agent_name']
#     )
    
#     prompt = f"あなたは{state['agent_name']}です。{state['other_agent_name']}と会話をしています。以下の会話を続けてください：\n\n{truncated_history}"
    
#     response = llm.invoke(prompt)
#     return {"messages": [AIMessage(content=response)]}

def generate_response(state: AgentState):
    truncated_history = get_truncated_conversation_history(
        state['messages'], 
        state['agent_name'], 
        state['other_agent_name']
    )
    
    prompt = f"""あなたは{state['agent_name']}です。{state['other_agent_name']}と会話をしています。
    以下のルールに従って会話を続けてください：
    1. 一貫性のある会話を維持してください。
    2. 突然話題を変えないでください。
    3. 前の会話の文脈を考慮して返答してください。
    4. 簡潔に、自然な会話をしてください。

    これまでの会話：
    {truncated_history}

    次の返答を生成してください："""
    
    response = llm.invoke(prompt)
    return {"messages": [AIMessage(content=response)]}

# 会話を管理するグラフを作成する関数
def create_conversation_graph(agent1_name: str, agent2_name: str):
    workflow = StateGraph(AgentState)
    
    workflow.add_node("agent1", lambda x: generate_response(x))
    workflow.add_node("agent2", lambda x: generate_response(x))
    
    workflow.set_entry_point("agent1")
    
    workflow.add_edge("agent1", "agent2")
    workflow.add_edge("agent2", "agent1")
    
    return workflow.compile()

# 会話グラフの作成
conversation = create_conversation_graph("Alice", "Bob")

# 会話を実行する関数
# def run_conversation(turns: int):
#     alice_state = create_agent_state("Alice", "Bob")
#     bob_state = create_agent_state("Bob", "Alice")
    
#     alice_state['messages'].append(HumanMessage(content="こんにちは、Bob。今日はいい天気ですね。散歩でもしませんか？"))
    
#     for _ in range(turns):
#         alice_result = conversation.invoke(alice_state)
#         alice_state['messages'].extend(alice_result['messages'])
#         bob_state['messages'].extend(alice_result['messages'])
        
#         bob_result = conversation.invoke(bob_state)
#         bob_state['messages'].extend(bob_result['messages'])
#         alice_state['messages'].extend(bob_result['messages'])
        
#         print(f"Alice: {alice_result['messages'][-1].content}")
#         print(f"Bob: {bob_result['messages'][-1].content}")
#         print()

def run_conversation(turns: int):
    alice_state = create_agent_state("Alice", "Bob")
    bob_state = create_agent_state("Bob", "Alice")
    
    alice_state['messages'].append(HumanMessage(content="こんにちは、Bob。今日はいい天気ですね。散歩でもしませんか？"))
    
    for _ in range(turns):
        alice_result = conversation.invoke(alice_state)
        alice_state['messages'].extend(alice_result['messages'])
        bob_state['messages'].extend(alice_result['messages'])
        
        bob_result = conversation.invoke(bob_state)
        bob_state['messages'].extend(bob_result['messages'])
        alice_state['messages'].extend(bob_result['messages'])
        
        print(f"Alice: {alice_result['messages'][-1].content}")
        print(f"Bob: {bob_result['messages'][-1].content}")
        print()


# 会話の実行
run_conversation(5)  # 5ターンの会話を実行




    回答する際には、前回の会話の内容や文脈を考慮し、自然な会話を継続します。


Bob:


    ※注意※


    会話が長時間にわたる場合、会話の流れや文脈を維持することが難しくなる可能性があります。会話の内容や文脈を適切に管理し、会話を自然かつ意味的に継続する努力をしてください。


Bob:


Alice: 
Alice: 



Bob:



Alice: 
Alice: 
Alice: 



Bob:


Alice: 



Alice: 



Alice: 



Bob:



Alice: 



Alice: 



Alice: 



Bob:



Alice: 



Alice: 



Alice: 



Bob:



Alice: 



Alice: 



Alice: 



Bob:



Alice: 



Alice: 



Alice: 



Bob:



Alice: 



Alice: 



Alice: 



Bob:



Alice: 



Alice: 



Alice: 



Bob:


    ※